In [ ]:
#%delete -f -s 251

In [1]:
from copy import deepcopy
import numpy as np
import pandas as pd
import re
import datetime
import gc
import pyspark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
207,application_1564590495923_4271,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
#%env JAVA_TOOL_OPTIONS:-Dfile.encoding=UTF16

In [4]:
%%info

An error was encountered:
Session of kind 'shared' not supported. Session must be of kinds spark, pyspark, pyspark3, sparkr.


In [57]:
text_data = sc.textFile('/user/aglushenko/head_encoded.xml', use_unicode=True)

In [9]:
tmp_list = create_xml_list(text_data)

In [333]:
text_data_full = sc.textFile('/user/aglushenko/012_SC_1_20180906_0001_utf.xml', use_unicode=True)

In [6]:
text_data_full.count()

292710766

In [ ]:
text_data_zipped = text_data_full.zipWithIndex()

In [283]:
begin_time = datetime.datetime.now()

first_xml = text_data_zipped.filter(lambda x: (x[1] > 2) & (x[1] <= 179)).keys().collect()

end_time = datetime.datetime.now()
exec_time = end_time - begin_time
print(exec_time)

0:03:11.738863

In [130]:
begin_time = datetime.datetime.now()

second_xml = text_data_zipped.filter(lambda x: (x[1] > 179) & (x[1] <= 766)).keys().collect()

end_time = datetime.datetime.now()
exec_time = end_time - begin_time
print(exec_time)

0:02:38.369118

In [9]:
print('\n'.join(second_xml))

<response num="120297472" dateofreport="01.09.2016" report_type="1234571467">
	<responsecode>1</responsecode>
	<responsestring>Клиент найден и у него есть кредитная история</responsestring>
	<title_part>
		<private>
			<lastname>СЕЛИВАНОВА</lastname>
			<firstname>ТАТЬЯНА</firstname>
			<middlename>НИКОЛАЕВНА</middlename>
			<birthday>11.10.1975</birthday>
			<birthplace>П ПУДОСТЬ ГАТЧИН. Р-Н ЛО</birthplace>
			<doc>
				<doctype>1</doctype>
				<docno>4100143407</docno>
				<docdate>30.08.2000</docdate>
				<docplace>ТАИЦКИМ ОМ ГАТЧИН. УВД ЛО</docplace>
			</doc>
			<history_title>
				<date>12.06.2018</date>
				<lastname>СЕЛИВАНОВА</lastname>
				<firstname>ТАТЬЯНА</firstname>
				<middlename>НИКОЛАЕВНА</middlename>
				<birthday>11.10.1975</birthday>
				<birthplace>ПОС. ПУДОСТЬ ГАТЧИНСКИЙ Р-Н ЛЕНИНГРАДСКАЯ ОБЛ.</birthplace>
				<doc>
					<doctype>1</doctype>
					<docno>4100143407</docno>
					<docdate>30.08.2000</docdate>
					<docplace>ТАИЦКИМ ОМ ГАТЧИНСКОГО УВД ЛО </docplace>


In [269]:
import xml.etree.ElementTree as ET
import collections

SEPARATOR = '_'

class XmlDictConfig(dict):
    '''
    Example usage:
    >>> tree = ElementTree.parse('your_file.xml')
    >>> root = tree.getroot()
    >>> xmldict = XmlDictConfig(root)
    Or, if you want to use an XML string:
    >>> root = ElementTree.XML(xml_string)
    >>> xmldict = XmlDictConfig(root)
    And then use xmldict for what it is... a dict.
    '''
    
    def __init__(self, parent_element, sep=SEPARATOR):
        if parent_element.attrib:
            self.update(parent_element.attrib)
        for element in parent_element:
            if len(element) >= 1:
                if len(element) == 1:
                    aDict = XmlDictConfig(element)
                else:
                    unique_element = ET.Element(element.tag)
                    
                    tags_seen = []
                    for sub_el in element:
                        if sub_el.tag not in tags_seen:
                            tags_seen.append(sub_el.tag)
                            if len(element.findall(sub_el.tag)) > 1: # if there are more than 1 elements with this tag 
                                
                                list_element = ET.Element(sub_el.tag)
                                
                                # find all elements with given tag and make them subelements of one element with the same tag
                                for el in element.findall(sub_el.tag):
                                    
                                    list_element.append(el)
                                    
                                try:
                                    aDict
                                except NameError:
                                    aDict = {list_element[0].tag: XmlListConfig(list_element)}
                                else:
                                    aDict.update({list_element[0].tag: XmlListConfig(list_element)})
                            else:
                                unique_element.append(sub_el)
                    try:
                        aDict
                    except NameError:
                        aDict = XmlDictConfig(unique_element)
                    else:
                        aDict.update(XmlDictConfig(unique_element))
                        
                if element.attrib:
                    aDict.update(element.attrib)
                self.update({element.tag: aDict})
                del aDict
            else:
                if element.text:
                    self.update({element.tag: element.text})
                if element.attrib:
                    self.update({element.tag + sep + k: v for k, v in element.items()})
                else:
                    self.update({element.tag: element.text})
                    
#     def __init__(self, parent_element, sep=SEPARATOR):
#         if parent_element.attrib:
#             self.update(parent_element.attrib)
#         for element in parent_element:
#             if len(element) >= 1:
#                 # treat like dict - we assume that if the first two tags
#                 # in a series are different, then they are all different.
#                 if len(element) == 1 or element[0].tag != element[1].tag:
#                     aDict = XmlDictConfig(element)
#                 # treat like list - we assume that if the first two tags
#                 # in a series are the same, then the rest are the same.
#                 else:
#                     aDict = {element[0].tag: XmlListConfig(element)}
#                 if element.attrib:
#                     aDict.update(element.attrib)
#                 self.update({element.tag: aDict})
#             else:
#                 if element.text:
#                     self.update({element.tag: element.text})
#                 if element.attrib:
#                     self.update({element.tag + sep + k: v for k, v in element.items()})
#                 else:
#                     self.update({element.tag: element.text})
    



class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if len(element) >= 1:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            else:
                lDict = {}
                if element.text:
                    text = element.text.strip()
                    if text:
                        lDict.update({'_VALUE': text})
                        if element.attrib:
                            lDict.update(element.attrib)
                elif element.attrib:
                    lDict.update(element.attrib)
                if len(lDict) > 0:
                    self.append(lDict)


# def flatten(d, parent_key='', sep=SEPARATOR):
#     items = []
#     for k, v in d.items():
#         new_key = parent_key + sep + k if parent_key else k
#         #print(new_key)
#         #if str(type(v)) == "<class 'XmlDictConfig'>" or (isinstance(v, dict)):
#         if str(type(v)) == "<class 'XmlDictConfig'>":
#             print(new_key)
#             #print(new_key)
#             items.extend(flatten(v, new_key, sep=sep).items())
#         elif (str(type(v)) == "<class 'XmlListConfig'>"):
#             print('list', new_key)
#             attr_arr = []
#             for el in v:
#                 print(type(el))
#                 if str(type(el)) == "<class 'XmlDictConfig'>":
#                     attr_arr.append(flatten(el, new_key, sep=sep))
#                 else:
#                     print((new_key, el))
#                     attr_arr.append({new_key: el})
#                 #print({new_key: attr_arr})
#             items.append((new_key, attr_arr))
#         else:
#             items.append((new_key, v))
#     return dict(items)

def flatten_row_type(d, parent_key='', sep=SEPARATOR):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        #print(new_key)
        if isinstance(v, pyspark.sql.types.Row) or isinstance(v, collections.MutableMapping):
            #print(new_key)
            items.extend(flatten_row_type(v.asDict(), new_key, sep=sep).items())
        elif isinstance(v, list):
            print('list', new_key)
            attr_arr = []
            for el in v:
                print(type(el))
                attr_arr.append(flatten_row_type(el.asDict(), new_key, sep=sep))
                #print({new_key: attr_arr})
            items.append((new_key, attr_arr))
        else:
            items.append((new_key, v))
    return dict(items)

def flatten(d, parent_key='', sep=SEPARATOR):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, list):
            flattened_elements = []
            for el in v:
                flattened_elements.append(flatten(el, new_key, sep))
            items.append((new_key, flattened_elements))
        elif isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


def parseXML(xml):
    if xml:
        tmp = flatten(XmlDictConfig(ET.fromstring(xml.encode('utf8'))))
        for a in tmp:
            if (type(tmp[a])) == XmlListConfig:
                tmp[a] = list(tmp[a])
        return tmp #.items()
    else:
        return []

In [270]:
second_xml_str = '\n'.join(second_xml)

In [271]:
second_xml_dict = XmlDictConfig(ET.XML(second_xml_str))

In [272]:
second_xml_dict

{'num': '120297472', 'dateofreport': '01.09.2016', 'report_type': '1234571467', 'responsecode': '1', 'responsestring': 'Клиент найден и у него есть кредитная история', 'title_part': {'private': {'history_title': [{'date': '12.06.2018', 'lastname': 'СЕЛИВАНОВА', 'firstname': 'ТАТЬЯНА', 'middlename': 'НИКОЛАЕВНА', 'birthday': '11.10.1975', 'birthplace': 'ПОС. ПУДОСТЬ ГАТЧИНСКИЙ Р-Н ЛЕНИНГРАДСКАЯ ОБЛ.', 'doc': {'doctype': '1', 'docno': '4100143407', 'docdate': '30.08.2000', 'docplace': 'ТАИЦКИМ ОМ ГАТЧИНСКОГО УВД ЛО '}, 'pfno': '05009766042'}, {'date': '09.08.2018', 'lastname': 'СЕЛИВАНОВА', 'firstname': 'ТАТЬЯНА', 'middlename': 'НИКОЛАЕВНА', 'birthday': '11.10.1975', 'birthplace': 'ПОС. ПУДОСТЬ ГАТЧИНСКИЙ Р-Н ЛЕНИНГРАДСКАЯ ОБЛ.', 'doc': {'doctype': '1', 'docno': '4100143407', 'docdate': '30.08.2000', 'docplace': '473-019'}, 'pfno': '05009766042'}, {'date': '13.08.2018', 'lastname': 'СЕЛИВАНОВА', 'firstname': 'ТАТЬЯНА', 'middlename': 'НИКОЛАЕВНА', 'birthday': '11.10.1975', 'birthplace': '

In [274]:
flat_second_xml_dict = flatten(second_xml_dict)

In [290]:
flat_second_xml_dict

{'num': '120297472', 'dateofreport': '01.09.2016', 'report_type': '1234571467', 'responsecode': '1', 'responsestring': 'Клиент найден и у него есть кредитная история', 'title_part_private_history_title': [{'title_part_private_history_title_date': '12.06.2018', 'title_part_private_history_title_lastname': 'СЕЛИВАНОВА', 'title_part_private_history_title_firstname': 'ТАТЬЯНА', 'title_part_private_history_title_middlename': 'НИКОЛАЕВНА', 'title_part_private_history_title_birthday': '11.10.1975', 'title_part_private_history_title_birthplace': 'ПОС. ПУДОСТЬ ГАТЧИНСКИЙ Р-Н ЛЕНИНГРАДСКАЯ ОБЛ.', 'title_part_private_history_title_doc_doctype': '1', 'title_part_private_history_title_doc_docno': '4100143407', 'title_part_private_history_title_doc_docdate': '30.08.2000', 'title_part_private_history_title_doc_docplace': 'ТАИЦКИМ ОМ ГАТЧИНСКОГО УВД ЛО ', 'title_part_private_history_title_pfno': '05009766042'}, {'title_part_private_history_title_date': '09.08.2018', 'title_part_private_history_title_l

In [275]:
def parse_to_spark_df(xml):
    
    #single_values = set()
    #multi_values = set()
    
    #for xml in xml_list:
        #single_xml_parsed = parseXML(xml)
    root = ET.XML(xml)
    xmldict = XmlDictConfig(root)
    single_xml_parsed = flatten(xmldict)

    single_values_dict = {}
    multi_values_dict = {}
    for k, v in single_xml_parsed.items():
        if type(v) != XmlListConfig:
            single_values_dict[k] = v
        else:
            multi_values_dict[k] = v

    #single_values = single_values.union(set(single_values_dict.keys()))
    #multi_values = multi_values.union(set(multi_values_dict.keys()))

    xml_df = pd.DataFrame.from_dict([single_values_dict])

    for k, v in multi_values_dict.items():
        if v:
            #print(v)
            xml_df[k] = [[dict(xml_dict) if str(type(xml_dict)) == "<class 'XmlDictConfig'>" 
                                         else xml_dict for xml_dict in v] if len(v) > 1 else v]
        else:
            xml_df[k] = []
    # if xml_df doesn't exist: create
    # else concatenate tmp_df with xml_df
#     try:
#         xml_df
#     except NameError:
#         xml_df = deepcopy(tmp_df)
#     else:
#         xml_df = pd.concat([xml_df, tmp_df])
            
    #To be able to trnsaform dataframe to spark dataframe we have to get rid of NaN values
    #Replace NaN with empty string
    for s_c in set(single_values_dict.keys()):
        xml_df.loc[xml_df[s_c].isnull(), s_c] = xml_df.loc[xml_df[s_c].isnull(), s_c].apply(lambda x: '')
    #Replace NaN with empty list
    for m_c in set(multi_values_dict.keys()):
        xml_df.loc[xml_df[m_c].isnull(), m_c] = xml_df.loc[xml_df[m_c].isnull(), m_c].apply(lambda x: [])
    
    return xml_df
    #return (spark.createDataFrame(xml_df), single_values_dict.keys(), multi_values_dict.keys())

In [277]:
spark_df_single_xml = spark.createDataFrame(parse_to_spark_df(second_xml_str))

In [281]:
personal_info_cols = ['title_part_private_firstname', 'title_part_private_middlename',
                      'title_part_private_lastname']

spark_df_single_xml = spark.createDataFrame(parse_to_spark_df(second_xml_str))
spark_df_single_xml.select(personal_info_cols).show()

+----------------------------+-----------------------------+---------------------------+
|title_part_private_firstname|title_part_private_middlename|title_part_private_lastname|
+----------------------------+-----------------------------+---------------------------+
|                     ТАТЬЯНА|                   НИКОЛАЕВНА|                 СЕЛИВАНОВА|
+----------------------------+-----------------------------+---------------------------+

In [280]:
spark_df_single_xml.columns

['add_part_attrs_package_id', 'add_part_attrs_package_name', 'add_part_attrs_package_section_id', 'add_part_attrs_package_section_name', 'add_part_attrs_package_section_period_attr', 'add_part_info_requests_request', 'add_part_interest_day', 'add_part_interest_hour', 'add_part_interest_month', 'add_part_interest_quarter', 'add_part_interest_week', 'add_part_interest_year', 'add_part_own_interest_day', 'add_part_own_interest_hour', 'add_part_own_interest_month', 'add_part_own_interest_quarter', 'add_part_own_interest_week', 'add_part_own_interest_year', 'base_part_addr_fact', 'base_part_addr_reg', 'base_part_credit', 'base_part_history_addr_addr_fact', 'base_part_history_addr_addr_reg', 'dateofreport', 'num', 'report_type', 'responsecode', 'responsestring', 'title_part_private_birthday', 'title_part_private_birthplace', 'title_part_private_doc_docdate', 'title_part_private_doc_docno', 'title_part_private_doc_docplace', 'title_part_private_doc_doctype', 'title_part_private_firstname', 't

In [285]:
first_xml_str = '\n'.join(first_xml)

spark_df_first_xml = spark.createDataFrame(parse_to_spark_df(first_xml_str))
spark_df_first_xml.select(personal_info_cols).show()

+----------------------------+-----------------------------+---------------------------+
|title_part_private_firstname|title_part_private_middlename|title_part_private_lastname|
+----------------------------+-----------------------------+---------------------------+
|                        ЮРИЙ|                   ДМИТРИЕВИЧ|                  КУЛИНЕНКО|
+----------------------------+-----------------------------+---------------------------+

In [286]:
spark_df_first_xml.columns

['add_part_attrs_package_id', 'add_part_attrs_package_name', 'add_part_attrs_package_section_id', 'add_part_attrs_package_section_name', 'add_part_attrs_package_section_period_attr_c_i', 'add_part_attrs_package_section_period_attr_n', 'add_part_attrs_package_section_period_attr_v', 'add_part_info_requests_request', 'add_part_interest_day', 'add_part_interest_hour', 'add_part_interest_month', 'add_part_interest_quarter', 'add_part_interest_week', 'add_part_interest_year', 'add_part_own_interest_day', 'add_part_own_interest_hour', 'add_part_own_interest_month', 'add_part_own_interest_quarter', 'add_part_own_interest_week', 'add_part_own_interest_year', 'base_part_addr_fact', 'base_part_addr_reg', 'base_part_credit_cred_active', 'base_part_credit_cred_active_date', 'base_part_credit_cred_collateral', 'base_part_credit_cred_currency', 'base_part_credit_cred_date', 'base_part_credit_cred_date_payout', 'base_part_credit_cred_day_overdue', 'base_part_credit_cred_enddate', 'base_part_credit_cr

In [287]:
spark_df_first_xml.union(spark_df_single_xml)

"Union can only be performed on tables with the same number of columns, but the first table has 73 columns and the second table has 38 columns;;\n'Union\n:- LogicalRDD [add_part_attrs_package_id#451, add_part_attrs_package_name#452, add_part_attrs_package_section_id#453, add_part_attrs_package_section_name#454, add_part_attrs_package_section_period_attr_c_i#455, add_part_attrs_package_section_period_attr_n#456, add_part_attrs_package_section_period_attr_v#457, add_part_info_requests_request#458, add_part_interest_day#459, add_part_interest_hour#460, add_part_interest_month#461, add_part_interest_quarter#462, add_part_interest_week#463, add_part_interest_year#464, add_part_own_interest_day#465, add_part_own_interest_hour#466, add_part_own_interest_month#467, add_part_own_interest_quarter#468, add_part_own_interest_week#469, add_part_own_interest_year#470, base_part_addr_fact#471, base_part_addr_reg#472, base_part_credit_cred_active#473, base_part_credit_cred_active_date#474, ... 49 more

In [289]:
set(spark_df_first_xml.columns) - set(spark_df_single_xml.columns)

{'base_part_credit_cred_date', 'base_part_credit_delay60', 'base_part_credit_cred_sum_type', 'base_part_credit_cred_sum_limit', 'base_part_credit_cred_sum', 'base_part_credit_delay90', 'base_part_credit_cred_first_load', 'base_part_credit_cred_active', 'base_part_credit_cred_update', 'add_part_attrs_package_section_period_attr_c_i', 'base_part_credit_delay_more', 'base_part_credit_cred_facility', 'base_part_credit_cred_id', 'add_part_attrs_package_section_period_attr_n', 'add_part_attrs_package_section_period_attr_v', 'base_part_credit_cred_active_date', 'base_part_credit_cred_prolong', 'base_part_credit_cred_partner_type', 'base_part_credit_cred_enddate', 'base_part_credit_delay30', 'base_part_credit_delay5', 'base_part_credit_op_cred_sum_paid', 'base_part_credit_cred_max_overdue', 'base_part_credit_cred_day_overdue', 'base_part_credit_cred_person_num', 'base_part_credit_ta_cred_sum_paid', 'base_part_credit_cred_sum_first', 'base_part_credit_cred_sum_payout', 'base_part_credit_cred_ow

In [328]:
response_pat = '^<response num='
response_close_pat = '^</response>$'

filtered = text_data_zipped.filter(lambda x: (x[1] > 2) & (x[1] < 100000) & \
                                             (True if re.search(response_close_pat, x[0]) else False))

In [329]:
response_close_ixs = filtered.values().collect()

In [330]:
print(len(response_close_ixs), response_close_ixs)

251 [178, 765, 983, 1246, 1956, 2364, 2538, 3095, 3200, 3348, 3922, 4523, 4917, 5156, 5325, 5599, 5935, 6131, 6446, 6761, 7367, 7496, 7904, 8318, 8585, 8887, 9204, 9475, 9929, 10174, 10465, 11072, 11590, 11850, 12303, 12549, 13108, 15323, 16106, 16364, 16764, 18320, 19420, 19614, 20292, 20729, 21591, 21761, 21994, 22628, 22780, 23302, 24405, 24920, 25306, 25765, 26275, 26465, 26766, 27343, 27623, 27802, 28713, 28841, 29105, 29402, 29995, 30052, 30357, 30519, 30985, 31548, 31975, 32050, 32129, 32372, 33154, 33711, 33814, 34279, 34898, 35181, 35460, 35788, 35941, 36423, 36668, 36998, 37940, 38131, 38524, 38642, 38991, 39263, 39444, 39885, 40236, 40765, 40870, 41444, 42017, 42535, 42707, 43331, 44117, 44729, 45286, 45508, 45740, 46000, 46213, 46405, 46962, 47248, 47563, 48234, 48463, 48973, 49214, 49771, 50134, 51537, 51884, 52313, 52604, 53113, 53487, 53973, 54506, 54809, 55067, 56024, 56404, 56927, 57078, 57300, 57761, 58469, 59052, 59462, 59661, 59847, 60082, 60608, 61139, 61579, 62223

In [331]:
first_xmls_strs = []
begin_ix = 2

for ix in response_close_ixs:
    xml =  text_data_zipped.filter(lambda x: (x[1] > begin_ix) & (x[1] <= ix + 1)).keys().collect()
    xml_str = '\n'.join(xml)
    
    first_xmls_strs.append(xml_str)
    
    begin_ix = ix + 1

len(first_xmls_strs)

251

In [ ]:
spark_df_first_xml = spark.createDataFrame(parse_to_spark_df(first_xml_str))
spark_df_first_xml.select(personal_info_cols).show()

In [315]:
pers_cols_df = spark_df_first_xml.select(personal_info_cols)

In [319]:
pers_cols_df.show()

+----------------------------+-----------------------------+---------------------------+
|title_part_private_firstname|title_part_private_middlename|title_part_private_lastname|
+----------------------------+-----------------------------+---------------------------+
|                        ЮРИЙ|                   ДМИТРИЕВИЧ|                  КУЛИНЕНКО|
+----------------------------+-----------------------------+---------------------------+

In [323]:
pers_cols_df = spark_df_first_xml.select(personal_info_cols)

for xml in first_xmls_strs:
    xml_spark_df = spark.createDataFrame(parse_to_spark_df(xml))
    pers_df = xml_spark_df.select(personal_info_cols)
    
    pers_cols_df = pers_cols_df.union(pers_df)
    
pers_cols_df.show()

0
1